# Projet UE MU4BM748

Antoine Auvergne <Br>
n° étudiant : 3522049

### Importation des bibliothèques

In [1]:
from tkinter import *
import numpy as np
import os
import re
import copy
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)
from matplotlib.figure import Figure

### Extraction des données
##### Protéomes

In [2]:
def extract_proteomes():
    os.chdir('./Proteomes')
    fichiers=!dir
    noms=[]
    
    for Fi in fichiers:
        if Fi[-4:]=='.faa' or Fi[-5:-1]=='.fasta':
            fichier=Fi.split(' ')[-1]                 #utile sous windows car le !dir fonctionnait différemment que
                                                      #sous linux
            noms.append('./Proteomes/'+fichier)
    os.chdir('..')
    data_proteomes={}
    for fichier in noms:
        motif=re.compile('[A-Za-z]{3}_[0-9]{9}[.]?[0-9]{0,2}')
        cle=motif.search(fichier).group()
        data_proteomes[cle]=[]
        lignes=open(fichier,'r')
        seq=''
        nomprot=''
        for ligne in lignes:
            if ligne[0]=='>':
                if seq!='' and nomprot!='':
                    data_proteomes[cle].append([nomprot,len(seq)*3,location[0],location[1]])
                seq=''
                nomprot=ligne.split(' ')[0][5:]
                location=(ligne.split('location=')[1]).split(']')[0].split('..')
                if location[0][0:4]=='comp':
                    location[0]=location[0].split('(')[1]
                    location[1]=location[1].split(')')[0]
            else:
                seq+=ligne.split('\n')[0]
        data_proteomes[cle].append([nomprot,len(seq)*3,location[0],location[1]])
        lignes.close()
    return(data_proteomes)
#data_proteomes=extract_proteomes()

##### CD search

In [3]:
def extract_cog_fun():
    fichier=open('cognames2003-2014.tab.txt','r',encoding='windows-1252')
    dico_cog={}
    for ligne in fichier:
        if ligne[0]!='#' and ligne[0]!='':
            dico_cog[ligne.split('\t')[0]]=(ligne.split('\t')[1],ligne.split('\t')[2].split('\n')[0])
    fichier=open('fun2003-2014.tab.txt','r',encoding='windows-1252')
    dico_fun={}
    for ligne in fichier:
        if ligne[0]!='#' and ligne[0]!='':
            dico_fun[ligne.split('\t')[0]]=ligne.split('\t')[1].split('\n')[0]
    return(dico_cog,dico_fun)

def extract_CDsearch():
    os.chdir('./CD-search')
    fichiers=!dir
    noms=[]
    for Fi in fichiers:
        if Fi[-4:]=='.out':
            fichier=Fi.split(' ')[-1]                 
            noms.append('./CD-search/'+fichier)
    os.chdir('..')
    data_cd={}
    for fichier in noms:
        motif=re.compile('[A-Za-z]{3}_[0-9]{9}[.]?[0-9]{0,2}')
        cle=motif.search(fichier).group()
        data_cd[cle]={}
        lignes=open(fichier,'r')
        for ligne in lignes:
            if ligne[0:2]=='Q#' and ligne.split('\t')[7][0:3]=='COG':
                compteur=int(ligne.split(' ')[0][2:])-1
                cle2=ligne.split(' ')[2][5:]
                ligne=ligne.split('\t')
                liste=[ligne[i] for i in [5,7]]
                liste.append(compteur)
                data_cd[cle][cle2]=liste
        lignes.close()
    return(data_cd)
                
                
#dico_cog,dico_fun=extract_cog_fun()
#data_cd=extract_CDsearch()
#print(data_cd['GCA_002843685.1']['CP025268.1_prot_AUG19080.1_4687'])

##### Blastp

In [4]:
def extract_blastp():
    os.chdir('./Blastp')
    fichiers=!dir
    noms=[]
    for Fi in fichiers:
        if Fi[-4:]=='.out':
            fichier=Fi.split(' ')[-1]
            noms.append('./Blastp/'+fichier)
    os.chdir('..')
    data_blastp={}
    for fichier in noms:
        motif=re.compile('[A-Za-z]{3}_[0-9]{9}[.]?[0-9]{0,2}')
        cle=motif.findall(fichier)
        cle=cle[0],cle[1]
        lignes=open(fichier,'r')
        data_blastp[cle]={}
        for ligne in lignes:
            if ligne[0]!='#':
                ligne=ligne.split('\t')
                nomprots=(ligne[0][4:],ligne[1])
                data_blastp[cle][nomprots]=ligne[2:-1]
          
        
        lignes.close()
    return(data_blastp)
#data_blastp=extract_blastp()

##### Prokaryotes complete genomes

In [5]:
def extract_compgenomes():
    fichier=open('prokaryotes_complete-genomes.csv','r')
    dico_genomes={}
    for ligne in fichier:
        if ligne[0]!='#':
            ligne=ligne.split(',')
            cle=ligne[5][1:-1]
            dico_genomes[cle]=[ligne[0][1:-1],ligne[1][1:-1],ligne[2][1:-1],ligne[7],ligne[8],ligne[12],ligne[14][1:-1],ligne[15][1:-2]]
    return(dico_genomes)

#compgenomes=extract_compgenomes()

### Récapitulatif des dictionnaires :
##### self.genomes : dictionnaire [ code GCA ] <Br>
    - indice 0 : Nom
    - indice 1 : Groupe Monophylétique
    - indice 2 : Espèce
    - indice 3 : Longueur du génome (en Mb)
    - indice 4 : %gc
    - indice 5 : CDS
    - indice 6 : FTP sur Genbank
    - indice 7 : FTPS sur RefSeq

##### self.cognames : dictionnaire [ code COG ]
    - indice 0 : Code à 1 lettre de la fonction
    - indice 1 : Nom
    
##### self.fonctioncog : dictionnaire [ code à une lettre de la fonction ]
    - renvoie la fonction associée à ce code
    
##### self.proteomes : dictionnaire [ code GCA ]
liste de k éléments (k étant le nombre de protéines pour le génome). Dans chaque élément, on a :
    - indice 0 : nom de la protéine
    - indice 1 : longueur de la protéine
    - indice 2 : début de la protéine sur le chromosome
    - indice 3 : fin de la protéine sur le chromosome
   
##### self.cd :  dictionnaire [ code GCA ] [ nom protéine ]
    - indice 0 : e-value associée au match
    - indice 1 : code COG
    - indice 2 : position dans le génome

##### self.blastp : dictionnaire [code GCA 1, code GCA 2] [nom protéine1, nom protéine2]
    - indice 0 : pourcentage d'identité
    - indice 1 : longueur du match
    - indice 2 : nombre de mismatch
    - indice 3 : nombre de gapopen
    - indice 4 : début du match sur la query
    - indice 5 : fin du match sur la query
    - indice 6 : début du match sur la séquence comparée
    - indice 7 : fin du match sur la séquence comparée
    - indice 8 : e-value associée au match

### Interface graphique

In [6]:
#Analyse de la syntenie
def syntenie(liste_coordonnees):
    #la fonction sort ne marche pas sur une array, on recrée donc une liste de liste
    coordlist=[]
    for i in range(len(liste_coordonnees)):
        liste=[]
        for j in range(len(liste_coordonnees[i])):
            liste.append(liste_coordonnees[i][j])
        coordlist.append(liste)
    blocs=[]
    dicodiag={}
    for coordonnee in coordlist:
        if coordonnee[0]-coordonnee[1] not in dicodiag.keys():
            dicodiag[coordonnee[0]-coordonnee[1]]=[]
        dicodiag[coordonnee[0]-coordonnee[1]].append(coordonnee)
    for diag in dicodiag.keys():
        if len(dicodiag[diag])>1:
            dicodiag[diag].sort()
            i=0
            while i<len(dicodiag[diag])-1:
                bloc_syntenie=[]
                bloc_syntenie.append(dicodiag[diag][i])
                i+=1
                while int(dicodiag[diag][i][0])==(int(dicodiag[diag][i-1][0])+1):
                    bloc_syntenie.append(dicodiag[diag][i])
                    i+=1
                    if i==len(dicodiag[diag]):
                        break
                if len(bloc_syntenie)>1:
                    blocs.append(bloc_syntenie)
    dicodiag={}
    for coordonnee in coordlist:
        if coordonnee[1]+coordonnee[0] not in dicodiag.keys():
            dicodiag[coordonnee[1]+coordonnee[0]]=[]
        dicodiag[coordonnee[1]+coordonnee[0]].append(coordonnee)
    blocsnega=[]
    for diag in dicodiag.keys():
        if len(dicodiag[diag])>1:
            dicodiag[diag].sort()
            i=0

            while i<len(dicodiag[diag])-1:
                bloc_syntenie=[]
                bloc_syntenie.append(dicodiag[diag][i])
                i+=1
                while int(dicodiag[diag][i][0])==(int(dicodiag[diag][i-1][0])+1):
                    bloc_syntenie.append(dicodiag[diag][i])
                    i+=1
                    if i==len(dicodiag[diag]):
                        break
                if len(bloc_syntenie)>1:
                    blocsnega.append(bloc_syntenie)
    return(blocs,blocsnega)

In [ ]:
class interface:
    def __init__(self):
        self.proteomes=extract_proteomes()
        self.cd=extract_CDsearch()
        self.blastp=extract_blastp()
        self.cognames,self.fonctioncog=extract_cog_fun()
        self.genomes=extract_compgenomes()
        self.stockimage={}
        if not os.path.exists('./Syntenies'):
            os.makedirs('./Syntenies')
        
        
        
        self.interface_para()
        
    def interface_para(self):
        
        # Les différents espaces à répétition dans les Label et autres checkbuttons permettent d'aligner ces différents
        # éléments entre eux. C'est également possible avec \t mais des carrés ('symboles introuvables') étaient alors affichés.
        
        self.fenetre=Tk()
        
        self.fenetre.title('Comparaison de paires de génomes')
        self.fenetre.geometry('900x500')
        
        # Création de la liste pour le menu déroulant
        self.listemenu=[]
        self.listecle=[]
        self.listegca=[]
        for i in self.blastp.keys():
            self.listegca.append(i[0])
            self.listegca.append(i[1])
            self.listemenu.append(self.genomes[i[0]][0]+' vs '+self.genomes[i[1]][0])
            self.listecle.append(self.genomes[i[0]][0])
            self.listecle.append(self.genomes[i[1]][0])
        
        # Choix des organismes à comparer
        self.description_1=Label(self.fenetre,text="\nOrganismes à comparer\n",font=('Helvetica',20))
        self.description_1.pack(side=TOP)
        self.initial=StringVar(self.fenetre)
        self.initial.set(self.listemenu[0])
        Choixorga=OptionMenu(self.fenetre,self.initial,*self.listemenu)
        Choixorga.config(width=90,font=('Helvetica',12))
        Choixorga.pack(side=TOP)
        
        # Choix des paramètres
        self.zonepara=Frame(self.fenetre)
        self.description_2=Label(self.zonepara,text="\nDéfinissez les paramètres de la comparaison\n",font=('Helvetica',20))
        self.description_2.pack(side=TOP)
        
        # Paramètre : pourcentage d'identité 
        self.pident_Frame=Frame(self.zonepara)
            # Définition des variables
        self.value_pident_binaire = IntVar()
        self.value_pident_binaire.set(0)
        
            # Frame de gauche, pour savoir si on tient compte de ce paramètre
        self.check_pident=Checkbutton(self.pident_Frame,text="Selon le pourcentage d'identité",font=('Helvetica',12),variable=self.value_pident_binaire)
        self.check_pident.pack(side=LEFT)
        
            # Frame de droite, pour déterminer un seuil
        self.pident_seuil_Frame=Frame(self.pident_Frame)
        self.pident_seuil_widget=Entry(self.pident_seuil_Frame)
        self.pident_seuil_indication=Label(self.pident_seuil_Frame,text="          seuil (entre 0 et 1, 0.2 par défaut) =",font=('Helvetica',12))
        self.pident_seuil_widget.pack(side=RIGHT)
        self.pident_seuil_indication.pack(side=LEFT)
        self.pident_seuil_Frame.pack(side=RIGHT)
        self.pident_Frame.pack(side=TOP)
        
        # Espace entre les paramètres
        self.espace1=Label(self.zonepara,text=" ")
        self.espace1.pack(side=TOP)

        # Paramètre : e-value
        
            #définition des variables
        self.value_blastp_binaire=IntVar()
        self.value_blastp_binaire.set(0)
        
        self.value_cd_binaire=IntVar()
        self.value_cd_binaire.set(0)
            
            # CD-search
        self.cd_Frame=Frame(self.zonepara)
        self.cd_Frame_choix=Frame(self.cd_Frame)
            # Frame de gauche, pour savoir si on tient compte de ce paramètre
        self.check_cd=Checkbutton(self.cd_Frame_choix, variable=self.value_cd_binaire)
        self.options=['Hit COG                                ','Annotation fonctionnelle     ']
        self.initial2=StringVar(self.cd_Frame_choix)
        self.initial2.set(self.options[0])
        self.Choixcd=OptionMenu(self.cd_Frame_choix,self.initial2,*self.options)
        self.check_cd.pack(side=LEFT)
        self.Choixcd.pack(side=RIGHT)
        self.cd_Frame_choix.pack(side=LEFT)
        
            # Frame de droite, pour déterminer un seuil
        self.cd_seuil_Frame=Frame(self.cd_Frame)
        self.cd_seuil_widget=Entry(self.cd_seuil_Frame)
        self.cd_seuil_indication=Label(self.cd_seuil_Frame,text="\t          seuil (1e-4 par défaut) =",font=('Helvetica',12))
        self.cd_seuil_widget.pack(side=RIGHT)
        self.cd_seuil_indication.pack(side=LEFT)
        self.cd_seuil_Frame.pack(side=RIGHT)
        self.cd_Frame.pack(side=TOP)
        
        # Espace entre les paramètres
        
        self.espace2=Label(self.zonepara,text=" ")
        self.espace2.pack(side=TOP)
        
            # Blastp
        self.bp_Frame=Frame(self.zonepara)
        
            # Pour savoir si on tient compte de ce paramètre
        self.check_bp=Checkbutton(self.bp_Frame,text="Selon la e-value du Blastp         ",font=('Helvetica',12),variable=self.value_blastp_binaire)
        self.check_bp.pack(side=LEFT)
        
            # Frame de droite, pour déterminer un seuil
        self.bp_seuil_Frame=Frame(self.bp_Frame)
        self.bp_seuil_widget=Entry(self.bp_seuil_Frame)
        self.bp_seuil_indication=Label(self.bp_seuil_Frame,text="\t          seuil (1e-4 par défaut) =",font=('Helvetica',12))
        self.bp_seuil_widget.pack(side=RIGHT)
        self.bp_seuil_indication.pack(side=LEFT)
        self.bp_seuil_Frame.pack(side=RIGHT)
        self.bp_Frame.pack(side=TOP)
        
        # Espace entre les paramètres      
        self.espace3=Label(self.zonepara,text=" ")
        self.espace3.pack(side=TOP)

        # Paramètre : couverture du hit
        self.hit_Frame=Frame(self.zonepara)
        self.value_hit_binaire = IntVar()
        self.value_hit_binaire.set(0)
        
        # Frame de gauche, pour savoir si on tient compte de ce paramètre
        self.check_hit=Checkbutton(self.hit_Frame,text="Selon la couverture du hit         ",font=('Helvetica',12),variable=self.value_hit_binaire)
        self.check_hit.pack(side=LEFT)
        
        # Frame de droite, pour déterminer un seuil
        self.hit_seuil_Frame=Frame(self.hit_Frame)
        self.hit_seuil_widget=Entry(self.hit_seuil_Frame)
        self.hit_seuil_indication=Label(self.hit_seuil_Frame,text="          seuil (entre 0 et 1, 0.2 par défaut) =",font=('Helvetica',12))
        self.hit_seuil_widget.pack(side=RIGHT)
        self.hit_seuil_indication.pack(side=LEFT)
        self.hit_seuil_Frame.pack(side=RIGHT)
        self.hit_Frame.pack(side=TOP)
                
        # Espace entre les paramètres
        self.espace4=Label(self.zonepara,text=" ")
        self.espace4.pack(side=TOP)
                
        # Bouton de fermeture du programme
        self.bouton_close=Button(self.zonepara,text="CLOSE",command=self.fenetre.destroy)
        self.bouton_close.pack(side=BOTTOM)
        
        # Bouton de lancement de l'analyse
        self.bouton_go=Button(self.zonepara,text="GO",command=self.interface_dotplot)
        self.bouton_go.pack(side=BOTTOM)
        
        self.zonepara.pack()
        self.fenetre.mainloop()
        
    def interface_dotplot(self):
        
        # Quels sont les organismes à comparer
        self.indicechoix=self.listemenu.index(self.initial.get())
        self.orga1,self.orga2=self.listecle[self.indicechoix*2],self.listecle[self.indicechoix*2+1]
        titre="Résultat de la comparaison des génomes de"+self.orga1+"et"+self.orga2
        #récupération des noms pour les axes du futur graphe.
        axe_x,axe_y=self.orga1,self.orga2        
        self.orga1,self.orga2=self.listegca[self.indicechoix*2],self.listegca[self.indicechoix*2+1]     #code GCA numéro 1 et 2    
        
        #test pour savoir si tous les fichiers ont pu être analysés
        if self.orga1 in self.proteomes.keys() and self.orga2 in self.proteomes.keys() and self.orga1 in self.cd.keys() and self.orga1 in self.cd.keys():
            #création des listes de nom de protéines dans l'ordre qui me permettront de savoir à quelles 
            #protéines correspondent des coordonnees (x,y)
            self.listeorga1=[]
            self.listeorga2=[]
            for i in range(len(self.proteomes[self.orga1])):
                self.listeorga1.append(self.proteomes[self.orga1][i][0])
            for i in range(len(self.proteomes[self.orga2])):
                self.listeorga2.append(self.proteomes[self.orga2][i][0])
            
            #Pour savoir par la suite si des donnees provenant de blast et/ou de CD-search sont utilisées
            #Il est important de les définir ici car sinon, elles ne seront pas définis si on n'utilise pas 
            #de données provenant d'un des fichiers, ce qui renverra une erreur lors de tests comme ceux utilisés
            #Pour la création du graphique (environ 30 ou 4 lignes plus bas)
            self.cd_bool=False
            self.blast_bool=False
            
            #Création de la fenêtre du graphe
            self.fenetre2=Tk()
            self.fenetre2.geometry('800x700')
            self.fenetre2.title(self.initial.get())
            
            #Création des listes de coordonnees qui nous intéressent
            self.creation_coor()
            
            #On indique sur la fenêtre du graphique les paramètres pris en compte
            paras="Paramètres pris en compte :"
            Aucun=True
            if self.value_blastp_binaire.get()==1:
                paras+="\nLa e-value associée au blastp avec un seuil de "+str(self.value_blastp_seuil)
                Aucun=False
            if self.value_pident_binaire.get()==1:
                paras+="\nLe pourcentage d'identité avec un seuil de "+str(self.value_pident_seuil*100)+"%"
                Aucun=False
            if self.value_hit_binaire.get()==1:
                paras+="\nLa couverture du hit avec un seuil de "+str(self.value_hit_seuil*100)+"%"
                Aucun=False
            if self.value_cd_binaire.get()==1:
                if self.indicechoixcd==0:
                    paras+="\nLa e-value associée au COG avec un seuil de "+str(self.value_cd_seuil)
                if self.indicechoixcd==1:
                    paras+="\nL'annotation fonctionnelle avec un seuil de "+str(self.value_cd_seuil)
                Aucun=False
            if Aucun==True:
                paras+="\nAucun"
            paras+="\n"
            label_para=Label(self.fenetre2,text=paras,font=('Helvetica',12))    
            label_para.pack(side=TOP)
            
            #Création du graphique sur la fenêtre 2 en fonction des fichiers utilisés
            fig=Figure(figsize=(5,3),dpi=96)
            self.ax=fig.add_subplot(111)
            if self.cd_bool==True:
                self.graphcd=self.ax.scatter(self.coordonnees_cd[:,0],self.coordonnees_cd[:,1],s=0.1,c='r',label="CD-search : Red")
            if self.blast_bool==True:
                self.graphblast=self.ax.scatter(self.coordonnees_blast[:,0],self.coordonnees_blast[:,1],s=0.1,c='b',label="Blastp : Blue")
            if self.cd_bool and self.blast_bool:
                self.coordonnees_both=[]
                for i in range(len(self.coordonnees_blast)):
                    if self.coordonnees_blast[i] in self.coordonnees_cd:
                        self.coordonnees_both.append(self.coordonnees_blast[i])
                self.coordonnees_both=np.array(self.coordonnees_both)
            self.ax.set_xlabel(axe_x)
            self.ax.set_ylabel(axe_y)
            self.ax.legend(loc='best')
 
            self.creation_syntenie()
            
            #cette variable permet de naviguer entre les différents dotplot sans effacer un dotplot qui 
            #n'existerait pas. Cette variable est utile pour les cas où on utilise les 2 fichiers mais,
            #afin de ne pas créer des fonctions quasi identiques, des tests liés à cette variable a été intégrée
            #aux fonction qui étaient utilisées pour afficher un seul dotplot. 
            #C'est pour ça qu'elle est définie ici.
            self.checkgraph=0
        
            #création des options supplémentaires pour le graphe
            self.frameoptions=Frame(self.fenetre2)
            self.afficher_synt=IntVar(self.fenetre2)
            self.afficher_synt.set(0)
            
            #Dans le cas où un seul des fichiers est utilisé, on a pas besoin de naviguer entre différents dotplot.
            if not self.blast_bool or not self.cd_bool:
                if self.blast_bool:
                    label_synt=Label(self.frameoptions,text="Synténie :",font=('Helvetica',12))
                    check_synt=Checkbutton(self.frameoptions,text="Afficher",font=('Helvetica',12),variable=self.afficher_synt)
                    self.afficher_synt.trace("w",self.Afficher_syntblast)
                    label_synt.pack(side=TOP)
                    check_synt.pack(side=TOP)
                elif self.cd_bool:
                    label_synt=Label(self.frameoptions,text="Synténie :",font=('Helvetica',12))
                    check_synt=Checkbutton(self.frameoptions,text="Afficher",font=('Helvetica',12),variable=self.afficher_synt)
                    self.afficher_synt.trace("w",self.Afficher_syntcd)
                    label_synt.pack(side=TOP)
                    check_synt.pack(side=TOP)
                
            
            #Dans le cas où les deux fichiers sont utilisés, il faut afficher d'autres options à l'écran
            if self.blast_bool and self.cd_bool:
                self.frameoptions1=Frame(self.frameoptions)
                self.frameoptions2=Frame(self.frameoptions)
                label_synt=Label(self.frameoptions2,text="Synténie :",font=('Helvetica',12))
                self.blast_binaire = IntVar(self.fenetre2)
                self.blast_binaire.set(1)
                self.blastaffich=Checkbutton(self.frameoptions1,text="Résultats du blast",font=('Helvetica',12),variable=self.blast_binaire)
                self.cd_binaire = IntVar(self.fenetre2)
                self.cd_binaire.set(1)
                self.cdaffich=Checkbutton(self.frameoptions1,text="Résultats du CD-search",font=('Helvetica',12),variable=self.cd_binaire)
                self.both_binaire = IntVar(self.fenetre2)
                self.both_binaire.set(0)
                self.bothaffich=Checkbutton(self.frameoptions1,text="Resultats identiques \nentre CD-search et Blastp",font=('Helvetica',12),variable=self.both_binaire)
                self.blastaffich.pack()
                self.cdaffich.pack()
                self.bothaffich.pack()
                self.frameoptions2=Frame(self.frameoptions)
                #les espaces dans cette liste permettent au menu déroulant de ne pas changer de taille lorsqu'on
                #change de choix. C'est également possible avec \t mais des carrés ('symboles introuvables') étaient
                #alors affichés.
                self.options=['Blastp                   ','CD-search              ','Intersection des deux']
                self.initial2=StringVar(self.fenetre2)
                self.initial2.set(self.options[0])
                Choixsynt=OptionMenu(self.frameoptions2,self.initial2,*self.options)
                check_synt=Checkbutton(self.frameoptions2,text="Afficher",font=('Helvetica',12),variable=self.afficher_synt)
                label_synt.pack(side=TOP)
                Choixsynt.pack(side=TOP)
                check_synt.pack(side=TOP)
                self.frameoptions1.pack(side=LEFT)
                self.frameoptions2.pack(side=RIGHT)
                # On veut suivre l'activation/la désactivation des checkbuttons en direct afin de mettre à jour le
                # dotplot en conséquence
                self.blast_binaire.trace('w',self.majblast)   
                self.cd_binaire.trace('w',self.majcd)
                self.both_binaire.trace('w',self.majboth) 
                self.afficher_synt.trace('w',self.Afficher_syntboth)
                self.initial.trace('w',self.Afficher_syntboth)
                
            # on propose un nom par défaut lors de la sauvegarde du graphique
            name=self.orga1+"vs"+self.orga2           
            self.graph=FigureCanvasTkAgg(fig, master=self.fenetre2)
            self.graph.get_default_filename = lambda: name
            self.graph.draw()
            self.graph.get_tk_widget().pack(side=TOP, fill=BOTH, expand=1)
            # Importation de la toolbar matplotlib 
            toolbar = NavigationToolbar2Tk(self.graph, self.fenetre2)
            toolbar.update()
            self.graph.get_tk_widget().pack(side=TOP, fill=BOTH, expand=1)
       
            framesaveclose=Frame(self.frameoptions)   
            if self.blast_bool or self.cd_bool:
                self.bouton_save=Button(framesaveclose,text="Sauvegarder la synténie",command=self.create_savefile)
                self.bouton_save.pack(side=RIGHT)
            self.bouton_close=Button(framesaveclose,text="CLOSE",command=self.fenetre2.destroy)
            self.bouton_close.pack(side=LEFT)
            framesaveclose.pack(side=BOTTOM)
            self.frameoptions.pack(side=BOTTOM) 

            
                    
            self.fenetre2.mainloop()
       
    #s'il manque un fichier, on l'indique
        else:
            if self.orga1 not in self.proteomes.keys():
                print("Il manque le fichier protéome de ",self.orga1)
            if self.orga2 not in self.proteomes.keys():
                print("Il manque le fichier protéome de ",self.orga2)
            if self.orga1 not in self.cd.keys():
                print("Il manque le fichier CD-search de ",self.orga1)
            if self.orga1 not in self.cd.keys():
                print("Il manque le fichier CD-search de ",self.orga2)
            #on ne vérifie pas le fichier blast car c'est à partir de ce fichier qu'on détermine 
            #quels organismes sont à comparer
    
    ################# Fonctions qui traitent les données
    
    ##### Calcul des coordonnees selon blast et/ou selon CD-search
    
    def creation_coor(self):
        #seuils de départ
        self.value_hit_seuil=0.2
        self.value_blastp_seuil=1e-4
        self.value_cd_seuil=1e-4
        self.value_pident_seuil=0.2
        motif_seuil=re.compile('[0-9]{1}e-[0-9]+')        
        motif_pourcentage=re.compile('0.[0-9]+')

        # Paramètre : cd-search
        if self.value_cd_binaire.get()==1:
            self.cd_bool=True
            if motif_seuil.match(self.cd_seuil_widget.get()):
                self.value_cd_seuil=float(self.cd_seuil_widget.get())
            self.indicechoixcd=self.options.index(self.initial2.get())
            self.coordonnees_cd=[]   
            #COG : au lieu de parcourir mes 2 listes pour voir si chaque protéine de l'organisme 1 match avec
            #chaque protéine de l'organisme 2 (ce qui peut faire plusieurs dizaines de millions de comparaisons)
            #on crée deux dictionnaires (un pour chaque organisme) dans lesquels les cog sont les clés. On peut
            #ainsi vérifier si un cog et présent dans chaque dictionnaire et si c'est le cas, on compare les
            #protéines 2 à 2.
            if self.indicechoixcd==0:
                dico_cdgroup={}
                dico_cdgroup[self.orga2]={}
                dico_cdgroup[self.orga1]={}
                for i in self.cd[self.orga1].keys():
                    if float(self.cd[self.orga1][i][0])<=self.value_cd_seuil:
                        if self.cd[self.orga1][i][1] not in dico_cdgroup[self.orga1].keys():
                            dico_cdgroup[self.orga1][self.cd[self.orga1][i][1]]=[]                   
                        dico_cdgroup[self.orga1][self.cd[self.orga1][i][1]].append(self.cd[self.orga1][i][2])
                for i in self.cd[self.orga2].keys():
                    if float(self.cd[self.orga2][i][0])<=self.value_cd_seuil:            
                        if self.cd[self.orga2][i][1] not in dico_cdgroup[self.orga2].keys():
                            dico_cdgroup[self.orga2][self.cd[self.orga2][i][1]]=[]                   
                        dico_cdgroup[self.orga2][self.cd[self.orga2][i][1]].append(self.cd[self.orga2][i][2])
                for codecog in dico_cdgroup[self.orga1].keys():
                    if codecog in dico_cdgroup[self.orga2].keys():
                        for element1 in dico_cdgroup[self.orga1][codecog]:
                            for element2 in dico_cdgroup[self.orga2][codecog]:
                                self.coordonnees_cd.append([element1,element2])
            elif self.indicechoixcd==1:
                #annotation fonctionnelle : pareil que pour cog
                dico_cdgroup={}
                dico_cdgroup[self.orga2]={}
                dico_cdgroup[self.orga1]={}
                for i in self.cd[self.orga1].keys():
                    if float(self.cd[self.orga1][i][0])<=self.value_cd_seuil:
                        annotation=self.cognames[self.cd[self.orga1][i][1]][0]
                        if len(annotation)==1:
                            if annotation not in dico_cdgroup[self.orga1].keys():
                                dico_cdgroup[self.orga1][annotation]=[]                   
                            dico_cdgroup[self.orga1][annotation].append(self.cd[self.orga1][i][2])
                        else:
                            for i2 in range(len(annotation)):
                                if annotation[i2] not in dico_cdgroup[self.orga1].keys():
                                    dico_cdgroup[self.orga1][annotation[i2]]=[]                   
                                dico_cdgroup[self.orga1][annotation[i2]].append(self.cd[self.orga1][i][2])
                for i in self.cd[self.orga2].keys():
                    if float(self.cd[self.orga2][i][0])<=self.value_cd_seuil:  
                        annotation=self.cognames[self.cd[self.orga2][i][1]][0]
                        if len(annotation)==1:
                            if annotation not in dico_cdgroup[self.orga2].keys():
                                dico_cdgroup[self.orga2][annotation]=[]                   
                            dico_cdgroup[self.orga2][annotation].append(self.cd[self.orga2][i][2])
                        else:
                            for i2 in range(len(annotation)):
                                if annotation[i2] not in dico_cdgroup[self.orga2].keys():
                                    dico_cdgroup[self.orga2][annotation[i2]]=[]                   
                                dico_cdgroup[self.orga2][annotation[i2]].append(self.cd[self.orga2][i][2])
                for annotation in dico_cdgroup[self.orga1].keys():
                    if annotation in dico_cdgroup[self.orga2].keys():
                        for element1 in dico_cdgroup[self.orga1][annotation]:
                            for element2 in dico_cdgroup[self.orga2][annotation]:
                                self.coordonnees_cd.append([element1,element2])
        if self.value_blastp_binaire.get()==1 or self.value_pident_binaire.get()==1 or self.value_hit_binaire.get()==1:
            self.blast_bool=True
            self.coordonnees_blast=[]

        # Paramètre : e-value du blastp
        if self.value_blastp_binaire.get()==1:
            if motif_seuil.match(self.bp_seuil_widget.get()):
                self.value_blastp_seuil=float(self.bp_seuil_widget.get())

            for couple in self.blastp[self.orga1,self.orga2].keys():
                if float(self.blastp[self.orga1,self.orga2][couple][8])<=self.value_blastp_seuil:
                    self.coordonnees_blast.append([self.listeorga1.index(couple[0]),self.listeorga2.index(couple[1])])

        # Paramètre : pourcentage d'identité
        if self.value_pident_binaire.get()==1:
            if motif_pourcentage.match(self.pident_seuil_widget.get()):
                self.value_pident_seuil=float(self.pident_seuil_widget.get())
            if not self.coordonnees_blast:
                for couple in self.blastp[self.orga1,self.orga2].keys():
                    if float(self.blastp[self.orga1,self.orga2][couple][0])/100>=self.value_pident_seuil:
                        self.coordonnees_blast.append([self.listeorga1.index(couple[0]),self.listeorga2.index(couple[1])])
            else:
                temp=[]
                for i in range(len(self.coordonnees_blast)):
                    xi=self.coordonnees_blast[i][0]
                    yi=self.coordonnees_blast[i][1]
                    compare_pourcentage=float(self.blastp[self.orga1,self.orga2][self.listeorga1[xi],self.listeorga2[yi]][0])
                    if compare_pourcentage/100>=self.value_pident_seuil:
                        temp.append(self.coordonnees_blast[i])
                self.coordonnees_blast=copy.deepcopy(temp)

        # Paramètre : couverture du hit
        if self.value_hit_binaire.get()==1:
            if motif_pourcentage.match(self.hit_seuil_widget.get()):
                self.value_hit_seuil=float(self.hit_seuil_widget.get())
            #Dans le cas où c'est le premier paramètre pris en compte
            if not self.coordonnees_blast:
                for couple in self.blastp[self.orga1,self.orga2].keys():
                    longueur_prot1=int(self.proteomes[self.orga1][self.listeorga1.index(couple[0])][1])
                    longueur_prot2=int(self.proteomes[self.orga2][self.listeorga2.index(couple[1])][1])
                    longueur_match1=int(self.blastp[self.orga1,self.orga2][couple][5])-int(self.blastp[self.orga1,self.orga2][couple][4])+1
                    longueur_match2=int(self.blastp[self.orga1,self.orga2][couple][7])-int(self.blastp[self.orga1,self.orga2][couple][6])+1
                    couverture1=(longueur_match1)/longueur_prot1
                    couverture2=(longueur_match2)/longueur_prot2
                    if couverture1>=self.value_hit_seuil and couverture2>=self.value_hit_seuil:
                        self.coordonnees_blast.append([self.listeorga1.index(couple[0]),self.listeorga2.index(couple[1])])
            #dans le cas où ça n'est pas le premier paramètre pris en compte
            else:
                temp=[]
                for i in range(len(self.coordonnees_blast)):
                    xi=self.coordonnees_blast[i][0]
                    yi=self.coordonnees_blast[i][1]
                    couple=(self.listeorga1[xi],self.listeorga2[yi])
                    if couple in self.blastp[self.orga1,self.orga2].keys():
                        longueur_prot1=int(self.proteomes[self.orga1][self.listeorga1.index(couple[0])][1])
                        longueur_prot2=int(self.proteomes[self.orga2][self.listeorga2.index(couple[1])][1])
                        longueur_match1=int(self.blastp[self.orga1,self.orga2][couple][5])-int(self.blastp[self.orga1,self.orga2][couple][4])+1
                        longueur_match2=int(self.blastp[self.orga1,self.orga2][couple][7])-int(self.blastp[self.orga1,self.orga2][couple][6])+1
                        couverture1=(longueur_match1)/longueur_prot1
                        couverture2=(longueur_match2)/longueur_prot2
                        if couverture1>=self.value_hit_seuil and couverture2>=self.value_hit_seuil:
                            temp.append(self.coordonnees_blast[i])
                self.coordonnees_blast=copy.deepcopy(temp)

        if self.blast_bool:
            self.coordonnees_blast=np.array(self.coordonnees_blast)
        if self.cd_bool:
            self.coordonnees_cd=np.array(self.coordonnees_cd)

    ##### Création des listes de coordonnée des blocs de synténie en fonction du/des fichiers utilisés, en 
    ##### utilisant la fonction synténie définit dans la cellule précédente
    
    def creation_syntenie(self):
        if self.cd_bool:
            self.syntcd_pos,self.syntcd_neg=syntenie(self.coordonnees_cd)
            self.coorsyntcd=[]
            for i in self.syntcd_pos:
                for j in i:
                    self.coorsyntcd.append(j)
            for i in self.syntcd_neg:
                for j in i:
                    self.coorsyntcd.append(j)
            self.coorsyntcd=np.array(self.coorsyntcd)
        if self.blast_bool:
            self.syntblast_pos,self.syntblast_neg=syntenie(self.coordonnees_blast)
            self.coorsyntblast=[]
            for i in self.syntblast_pos:
                for j in i:
                    self.coorsyntblast.append(j)
            for i in self.syntblast_neg:
                for j in i:
                    self.coorsyntblast.append(j)
            self.coorsyntblast=np.array(self.coorsyntblast)
        if self.blast_bool and self.cd_bool:
            self.syntboth_pos,self.syntboth_neg=syntenie(self.coordonnees_both)
            self.coorsyntboth=[]
            for i in self.syntboth_pos:
                for j in i:
                    self.coorsyntboth.append(j)
            for i in self.syntblast_neg:
                for j in i:
                    self.coorsyntboth.append(j)
            self.coorsyntboth=np.array(self.coorsyntboth)
    
    ##### mise à jour de l'affichage des listes de coordonnees       
            
    def majcd(self,*args):
        if self.cd_binaire.get()==0:
            self.graphcd.remove()
            self.ax.get_legend().remove()
            if self.blast_binaire.get()==1:
                self.ax.legend(loc='best')
            self.graph.draw()
        if self.cd_binaire.get()==1:
            if self.both_binaire.get()==1:
                self.both_binaire.set(0)
            self.graphcd=self.ax.scatter(self.coordonnees_cd[:,0],self.coordonnees_cd[:,1],s=0.1,c='r',label="CD-search : Red")
            self.ax.legend(loc='best')
            self.graph.draw()

    def majblast(self,*args):
        if self.blast_binaire.get()==0:
            self.graphblast.remove()
            self.ax.get_legend().remove()
            if self.cd_binaire.get()==1:
                self.ax.legend(loc='best')
            self.graph.draw()
        if self.blast_binaire.get()==1:
            if self.both_binaire.get()==1:
                self.both_binaire.set(0)
            self.graphblast=self.ax.scatter(self.coordonnees_blast[:,0],self.coordonnees_blast[:,1],s=0.1,c='b',label="Blastp : Blue")
            self.ax.legend(loc='best')
            self.graph.draw()

    def majboth(self,*args):
        if self.both_binaire.get()==0:
            self.graphboth.remove()
            self.blast_binaire.set(1)
            self.cd_binaire.set(1)
        if self.both_binaire.get()==1:
            if self.blast_binaire.get()==1:
                self.blast_binaire.set(0)
            if self.cd_binaire.get()==1:
                self.cd_binaire.set(0)
            self.graphboth=self.ax.scatter(self.coordonnees_both[:,0],self.coordonnees_both[:,1],s=0.1,c='c',label="Intersection \ndes deux : Cyan")
            self.ax.legend(loc='best')
            self.graph.draw()

    ##### mise à jour de l'affichage des blocs de synténie

    def Afficher_syntblast(self,*args):
        if self.afficher_synt.get()==0:
            if self.checkgraph==1:
                self.graphsynt.remove()
                self.graph.draw()
            self.checkgraph=0
        if self.afficher_synt.get()==1:
            if self.checkgraph==1:
                self.graphsynt.remove()
            self.graphsynt=self.ax.scatter(self.coorsyntblast[:,0],self.coorsyntblast[:,1],s=0.1,c='k',label="Synténie: Noir")
            self.ax.legend(loc='best')
            self.graph.draw()
            self.checkgraph=1

    def Afficher_syntcd(self,*args):
        if self.afficher_synt.get()==0:
            if self.checkgraph==1:
                self.graphsynt.remove()
                self.graph.draw()
            self.checkgraph=0
        if self.afficher_synt.get()==1:
            if self.checkgraph==1:
                self.graphsynt.remove()
            self.graphsynt=self.ax.scatter(self.coorsyntcd[:,0],self.coorsyntcd[:,1],s=0.1,c='k',label="Synténie: Noir")
            self.ax.legend(loc='best')
            self.graph.draw()
            self.checkgraph=1

    def Afficher_syntboth(self,*args):
        if self.afficher_synt.get()==0:
            if self.checkgraph==1:
                self.graphsynt.remove()
                self.graph.draw()
            self.checkgraph=0
        if self.afficher_synt.get()==1:
            ind=self.options.index(self.initial2.get())
            if ind==0:
                self.Afficher_syntblast()
            if ind==1:
                self.Afficher_syntcd()
            if ind==2:
                if self.checkgraph==1:
                    self.graphsynt.remove()
                self.graphsynt=self.ax.scatter(self.coorsyntboth[:,0],self.coorsyntboth[:,1],s=0.1,c='k',label="Synténie: Noir")
                self.ax.legend(loc='best')
                self.graph.draw()
            self.checkgraph=1
    
    ##### sauvegarde des blocs de synténie sélectionnés
    
    def create_savefile(self):
        self.fenetresave=Tk()
        self.fenetresave.title("Sauvegarde")
        self.fenetresave.geometry('300x100')
        label=Label(self.fenetresave,text='Sauvegarder sous quel nom ? Tout fichier préexistant sera écrasé.',font=('Helvetica',12))
        self.save_widget=Entry(self.fenetresave)
        framebutton=Frame(self.fenetresave)
        buttonsave=Button(framebutton,text="Sauvegarder",command=self.Save_synte)
        buttoncancel=Button(framebutton,text="Annuler",command=self.fenetresave.destroy)
        buttonsave.pack(side=LEFT)
        buttoncancel.pack(side=RIGHT)
        framebutton.pack(side=BOTTOM)
        label.pack(side=TOP)
        self.save_widget.pack(side=TOP)
        self.fenetresave.mainloop()
        
    def Save_synte(self):
        nomFi='./Syntenies/'+self.orga1+"vs"+self.orga2
        nom=str(self.save_widget.get())
        if nom!='':
            nomFi='./Syntenies/'+self.save_widget.get()
        fichier=open(nomFi,'w')
        fichier.write('#Blocs de synténie entre '+self.orga1+' et '+self.orga2+'\n#\n######################\n#Paramètres pris en compte :\n')
        if self.cd_bool:
            paracd='#* Fichier CD-search :\n'
            if self.indicechoixcd==0:
                paracd+='#\t-> E-value associée aux COG avec un seuil de '+str(self.value_cd_seuil)+'\n'
            if self.indicechoixcd==1:
                paracd+='#\t-> Annotation fonctionnelle avec un seuil de '+str(self.value_cd_seuil)+'\n'
        if self.blast_bool:
            parablast='#* Fichier Blastp :\n'
            if self.value_blastp_binaire.get()==1:
                parablast+='#\t-> E-value associée au blastp avec un seuil de '+str(self.value_blastp_seuil)+'\n'
            if self.value_pident_binaire.get()==1:
                parablast+='#\t-> E-value associée au blastp avec un seuil de '+str(self.value_pident_seuil)+'\n'
            if self.value_hit_binaire.get()==1:
                parablast+='#\t-> E-value associée au blastp avec un seuil de '+str(self.value_hit_seuil)+'\n'
        if self.cd_bool and self.blast_bool:
            if self.options.index(self.initial2.get())==0:
                listepos,listeneg=self.syntblast_pos,self.syntblast_neg
                fichier.write(parablast)
            if self.options.index(self.initial2.get())==1:
                listepos,listeneg=self.syntcd_pos,self.syntcd_neg
                fichier.write(paracd)
            if self.options.index(self.initial2.get())==2:
                listepos,listeneg=self.syntboth_pos,self.syntboth_neg
                fichier.write(paracd+parablast)
        elif self.cd_bool:
            listepos,listeneg=self.syntcd_pos,self.syntcd_neg
            fichier.write(paracd)
        elif self.blast_bool:
            listepos,listeneg=self.syntblast_pos,self.syntblast_neg
            fichier.write(parablast)
        fichier.write('######################\n\n> Blocs qui sont dans le bon sens\n#start_name_orga1\tend_name_orga1\tstart_loc_orga1\tend_loc_orga1\tstart_name_orga2\tend_name_orga2\tstart_loc_orga2\tend_loc_orga2\tNb_prot\n')
        for i in listepos:
            indstart1=i[0][0]
            indstart2=i[0][1]
            indend1=i[-1][0]
            indend2=i[-1][1]
            bloc=self.listeorga1[indstart1]+'\t'+self.listeorga1[indend1]+'\t'+self.proteomes[self.orga1][indstart1][2]+'\t'+self.proteomes[self.orga1][indend1][3]+'\t'+self.listeorga2[indstart2]+'\t'+self.listeorga2[indend2]+'\t'+self.proteomes[self.orga2][indstart2][2]+'\t'+self.proteomes[self.orga2][indend2][3]+'\t'+str(len(i))+'\n'
            fichier.write(bloc)
        fichier.write('######################\n\n> Blocs qui sont dans le mauvais sens\n#start_name_orga1\tend_name_orga1\tstart_loc_orga1\tend_loc_orga1\tstart_name_orga2\tend_name_orga2\tstart_loc_orga2\tend_loc_orga2\tNb_prot\n')
        print(len(self.listeorga1))
        for i in listeneg:
            indstart1=i[0][0]
            indstart2=i[-1][1]
            indend1=i[-1][0]
            indend2=i[0][1]
            bloc=self.listeorga1[indstart1]+'\t'+self.listeorga1[indend1]+'\t'+self.proteomes[self.orga1][indstart1][2]+'\t'+self.proteomes[self.orga1][indend1][3]+'\t'+self.listeorga2[indstart2]+'\t'+self.listeorga2[indend2]+'\t'+self.proteomes[self.orga2][indstart2][2]+'\t'+self.proteomes[self.orga2][indend2][3]+'\t'+str(len(i))+'\n'
            fichier.write(bloc)
        fichier.close()
        self.fenetresave.destroy() 
        
a=interface()

        

4137
